In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('Healthcare_dataset.xlsx', engine='openpyxl', sheet_name='Dataset', na_values=['Unknown', 'Other/Unknown'])

In [3]:
# Counting rows with NA
na_count = data.shape[0] - data.dropna().shape[0]
percent_na = na_count / data.shape[0] * 100
print("Percent of rows with NAs = ", percent_na, "%")

Percent of rows with NAs =  69.50934579439252 %


In [4]:
#Extract rows with NA values only
nan_df = data[data.isna().any(axis=1)]
#Count frequency of persistency flag
persist_na = nan_df['Persistency_Flag'].value_counts()['Persistent']
persist_tot = data['Persistency_Flag'].value_counts()['Persistent']

print('How many persistent flags will be removed by deletion method: ', persist_na / persist_tot * 100, '%')

How many persistent flags will be removed by deletion method:  64.70131885182312 %


In [5]:
#Map values to numbers
columns_map = {}
for col in data.columns:
    if str(data[col].dtype) == 'object':  #If it's a categorical variable
        val_map = {}
        mapped_value = 0
        for idx, val in enumerate(data[col].unique()): 
            not_na = True
            if type(val) == float:
                not_na = (np.isnan(val) != True)
            if  not_na == True: #If the value is not NaN add it to the mapping
                val_map[val] = mapped_value 
                mapped_value += 1
            columns_map[col] = val_map
            
print(columns_map['Ethnicity'])

{'Not Hispanic': 0, 'Hispanic': 1}


In [8]:
#dealing with missing data
from sklearn.impute import KNNImputer

df = data.copy()
#Using Mapping to encode data
for col in df.columns:
    if str(df[col].dtype) == 'object':
        df[col] = df[col].map(columns_map[col])
   
df_filled = df.copy()
imputer= KNNImputer(n_neighbors=5)
df_filled = imputer.fit_transform(df_filled)
df_filled = pd.DataFrame(df_filled, columns=df.columns).apply(round)
df_filled.isna().any().any()

False